# Catbooster

## Meta preparation

In [3]:
# Environment settings.
is_kaggle = False

In [4]:
# Load libraries.
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1"
import pandas as pd, numpy as np
import matplotlib.pyplot as pyplot

VER = 2

## Train data preparation

In [5]:
train_csv_path = "./../data/input/train.csv" if not is_kaggle else "/kaggle/input/hms-harmful-brain-activity-classification/train.csv"
train_df = pd.read_csv(train_csv_path)
targets = train_df.columns[-6:]

print("Train shape: ", train_df.shape)
print("Targets: ", list(targets))

train_df.head()

Train shape:  (106800, 15)
Targets:  ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


### Create Non-Overlapping Eeg Id Train Data

The competition data description says that test data does not have multiple crops from the same eeg_id. Therefore we will train and validate using only 1 crop per eeg_id. There is a discussion about this [discussion](https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021).

In [15]:
train_df_grouped_by_eeg_id = train_df.groupby("eeg_id")

train_df_agg = train_df_grouped_by_eeg_id[["spectrogram_id", "spectrogram_label_offset_seconds"]].agg(
    {"spectrogram_id": "first",
     "spectrogram_label_offset_seconds": "min"}
)
train_df_agg.columns = ["spec_id", "min"]

tmp = train_df_grouped_by_eeg_id[["spectrogram_id", "spectrogram_label_offset_seconds"]].agg(
    {"spectrogram_label_offset_seconds": "max"}
)
train_df_agg['max'] = tmp

tmp = train_df_grouped_by_eeg_id[["patient_id"]].agg("first")
train_df_agg['patient_id'] = tmp

tmp = train_df_grouped_by_eeg_id[targets].agg("sum")
for target in targets:
    train_df_agg[target] = tmp[target].values

tmp = train_df_grouped_by_eeg_id[["expert_consensus"]].agg("first")
train_df_agg["target"] = tmp

train_df_agg = train_df_agg.reset_index()
print("Train non-overlapped eeg_id shape: ", train_df_agg.shape)

train_df_agg.head()

Train non-overlapped eeg_id shape:  (17089, 12)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0,0,12,0,8,28,Other
1,582999,1552638400,0.0,38.0,20230,0,132,0,11,0,11,LPD
2,642382,14960202,1008.0,1032.0,5955,0,0,0,0,0,2,Other
3,751790,618728447,908.0,908.0,38549,0,0,1,0,0,0,GPD
4,778705,52296320,0.0,0.0,40955,0,0,0,0,0,2,Other


### Feature Engineer
First we need to read in all 11k train spectrogram files. Reading thousands of files takes 11 minutes with Pandas. Instead, we can read 1 file from [Brain-Spectrograms dataset](https://www.kaggle.com/datasets/cdeotte/brain-spectrograms), which contains all the 11k spectrograms in less than 1 minute! To use this dataset, set variable READ_SPEC_FILES = False.

Next we need to engineer features for our CatBoost model. In this notebook, we just take the mean (over time) of each of the 400 spectrogram frequencies (using middle 10 minutes). This produces 400 features (per each unique eeg id). We can improve CV and LB score by engineering new features (and/or tuning CatBoost).

Also here we create features from means and mins and use 10 minute windows and 20 second windows.

In [16]:
READ_SPEC_FILES = False
FEATURE_ENGINEER = True

In [17]:
%%time
# Read all spectrograms.
spectrograms_dir_path = "./../data/input/train_spectrograms"
spectrogram_files = os.listdir(spectrograms_dir_path)

print(f"There are {(len(spectrogram_files))} spectrogram parquets")

if READ_SPEC_FILES:
    spectrograms = {}
    for index, spectrogram_file in enumerate(spectrogram_files):
        if index % 100 == 0:
            print(f"{index}, ", end="")
        tmp = pd.read_parquet(f"{spectrograms_dir_path}{spectrogram_file}")
        name = int(spectrogram_file.split(".")[0])
        spectrograms[name] = tmp.iloc[:, 1:].values
else:
    spectrograms_npy_path = "./../data/input/brain-spectrograms/specs.npy" if not is_kaggle else "kaggle/input/brain-spectrograms/specs.npy"
    spectrograms = np.load(spectrograms_npy_path, allow_pickle=True).item()

There are 11138 spectrogram parquets
CPU times: user 644 ms, sys: 12.5 s, total: 13.2 s
Wall time: 16.8 s


In [27]:
%time
# Engineer features.
import warnings
warnings.filterwarnings("ignore")

spec_cols = pd.read_parquet(os.path.join(spectrograms_dir_path, "353733.parquet")).columns[1:]
features = [f"{col}_mean_10m" for col in spec_cols]
features += [f"{col}_min_10m" for col in spec_cols]
features += [f"{col}_mean_20m" for col in spec_cols]
features += [f"{col}_min_20m" for col in spec_cols]

print(f"We are creating {len(features)} features for {len(train_df_agg)} rows...", end="")

if FEATURE_ENGINEER:
    data = np.zeros((len(train_df_agg), len(features)))
    for k in range(len(train_df_agg)):
        if k % 100 == 0:
            print(f"{k}, ", end="")
        
        row = train_df_agg.iloc[k]
        r = int(row['min'] + row['max'] // 4)
        
        # 10 minute window features (means and mins)
        x = np.nanmean(spectrograms[row.spec_id][r:r+300, :], axis=0)
        data[k, :400] = x
        x = np.nanmin(spectrograms[row.spec_id][r:r+300, :], axis=0)
        data[k, 400:800] = x
        
        # 20 second window features (means and mins)
        x = np.nanmean(spectrograms[row.spec_id][r+145:r+155, :], axis=0)
        data[k, 800:1200] = x
        x = np.nanmin(spectrograms[row.spec_id][r+145:r+155, :], axis=0)
        data[k, 1200:1600] = x
        
    train_df_agg[features] = x
else:
    train_pqt_path = "./../data/input/brain-spectrograms/train.pqt" if not is_kaggle else "kaggle/input/brain-spectrograms/train.pqt"
    train_df_agg = pd.read_parquet(train_pqt_path)

print()
print("New train shape: ", train_df_agg.shape)

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 5.48 µs
We are creating 1600 features for 17089 rows...0, [3.9260e+01 3.9740e+01 6.3400e+01 6.1630e+01 4.3550e+01 3.6440e+01
 2.7520e+01 1.8510e+01 2.9470e+01 2.6750e+01 1.6040e+01 1.1290e+01
 1.1660e+01 1.1430e+01 1.1130e+01 7.0200e+00 9.6800e+00 1.0590e+01
 9.4900e+00 1.0110e+01 9.0000e+00 8.4400e+00 6.7400e+00 7.2900e+00
 6.4800e+00 5.7000e+00 4.8300e+00 3.3400e+00 3.5400e+00 3.4800e+00
 3.0100e+00 3.1600e+00 2.5400e+00 1.7300e+00 1.6400e+00 1.5300e+00
 1.4200e+00 1.5000e+00 1.2500e+00 1.1300e+00 8.3000e-01 1.0400e+00
 8.3000e-01 9.5000e-01 8.6000e-01 7.7000e-01 7.0000e-01 6.2000e-01
 5.1000e-01 4.1000e-01 4.7000e-01 4.3000e-01 4.3000e-01 2.4000e-01
 2.8000e-01 2.8000e-01 2.4000e-01 2.4000e-01 2.6000e-01 2.3000e-01
 2.4000e-01 2.3000e-01 2.4000e-01 2.2000e-01 2.0000e-01 1.8000e-01
 1.5000e-01 1.2000e-01 1.0000e-01 1.4000e-01 1.2000e-01 1.5000e-01
 1.1000e-01 1.3000e-01 1.3000e-01 1.4000e-01 1.2000e-01 1.2000e-01
 1.0000e-01 1.2

ValueError: zero-size array to reduction operation fmin which has no identity

## References
- [CatBoost Starter](https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-67)